### Word Mover's Distance Implementation by cvxopt

In [1]:
#pip install nltk

In [5]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# 读取Glove文件。 注意： 不要试图修改文件以及路径
#glovefile = open("glove.6B.100d.txt","r",encoding="utf-8")  


# TODO: 编写WMD函数来计算两个句子之间的相似度
def convert2freq(wdlist):
    """
    input: a list of words.
    return : a dictionary of unique words and the corresponding frequency.
    """
    d = {}
    for wd in wdlist:
        if wd in d:
            d[wd.lower()] += 1
        else:
            d[wd.lower()] = 1
    
    return d

def sent_pre_1(sent, flag):
    """
    input1 : sentence
    input2 : True = remove the stop words; False = otherwise
    output : a dictionary of word frequency
    """
    # get the word list from sentense
    if flag:
        text_tokens = tokenizer.tokenize(sent)
        wdlist = [word for word in text_tokens if not word in stopwords.words()]
    else:
        wdlist = sent.strip().split()
     
    wddict = convert2freq(wdlist)
    return wddict
    
def sent_pre_2(wdmatrix, wddict):
    """
    input : word2vec matrix of wdlist
    output : v, d. prepare the input for MWD.
    """
    d, v = [], []
    for item in wdmatrix:
        d.append(wddict[item[0]])
        v.append(item[0])
    total = sum(d)
    d = np.array(d)
    d = d/total   
    return d, v
    
        
def WMD (sent1, sent2, flag):
    """
    这是主要的函数模块。参数sent1是第一个句子， 参数sent2是第二个句子，可以认为没有经过分词。
    在英文里，用空格作为分词符号。
    
    在实现WMD算法的时候，需要用到LP Solver用来解决Transportation proboem. 请使用http://cvxopt.org/examples/tutorial/lp.html
    也可以参考blog： https://scaron.info/blog/linear-programming-in-python-with-cvxopt.html
    
    需要做的事情为：
    
    1. 对句子做分词： 调用 .split() 函数即可
    2. 获取每个单词的词向量。这需要读取文件之后构建embedding matrix. 
    3. 构建lp问题，并用solver解决
    
    可以自行定义其他的函数，但务必不要改写WMD函数名。测试时保证WMD函数能够正确运行。
    """
    wddict1 = sent_pre_1(sent1, flag)
    wddict2 = sent_pre_1(sent2, flag)
    #print(wddict1)
    #print(wddict2)
    
    # fetch the word2vec from glove database
    wdmatrix1 = []
    wdmatrix2 = []
    
    glovefile = open("glove.6B.100d.txt","r",encoding="utf-8")  
    for line in glovefile.readlines():
        current = line.split()
        #print(current[0])
        if current[0] in wddict1:
            wdmatrix1.append(current)
        if current[0] in wddict2:
            wdmatrix2.append(current)    
    glovefile.close()
    
    # calculate d1 and d2
    
    d1, v1 = sent_pre_2(wdmatrix1, wddict1)
    d2, v2 = sent_pre_2(wdmatrix2, wddict2)
    #print(d1, d2)
    #print(v1, v2)
    
    # calculate c(i, j)
    n = len(wddict1)
    m = len(wddict2)
    
    embedding = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            xi = np.array(wdmatrix1[i][1:])
            xi = xi.astype(np.float)
            xj = np.array(wdmatrix2[j][1:])
            xj = xj.astype(np.float)
            embedding[i][j] = np.linalg.norm(xi - xj)
    #print(embedding)
    
    # optimization in calculation WMD
    c = embedding.flatten().tolist()
    #print(c)
    
    # construct A
    # the contrain in i
    Ai_total = []
    for i in range(n):
        Ai = np.zeros((n, m))
        Ai[i,:] = 1
        Ai_total.append(Ai.flatten())
    Ai_total = np.array(Ai_total)
    #print('Ai')
    #print(Ai_total)
    
    # the contain in j
    Aj_total = []
    for j in range(m):
        Aj = np.zeros((n, m))
        Aj[:,j] = 1
        Aj_total.append(Aj.flatten())
    Aj_total = np.array(Aj_total)
    #print('Aj')
    #print(Aj_total)
    
    # positive entries
    A_total = []
    for i in range(n):
        for j in range(m):
            A = np.zeros((n, m))
            A[i,j] = -1
            A_total.append(A.flatten())
            
    A = np.concatenate((Ai_total, Aj_total, -Ai_total, -Aj_total, A_total))
    A = (A.T).tolist()
    #print(A)
    
    # construct b
    b = np.concatenate((d1, d2, -d1, -d2, np.zeros(n*m))).tolist()
    #print(b)
    
    # do optimization
    from cvxopt import matrix, solvers
    opA = matrix(A)
    opb = matrix(b)
    opc = matrix (c)
    sol = solvers.lp(opc, opA, opb)
    return sol['primal objective']
    
    
    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# print out the stop words
print(set(stopwords.words('english')))

{'are', 'have', 'it', 'its', 's', 'wouldn', 'weren', 'now', 'all', 'aren', 'hasn', 'same', 'your', 'were', 'further', 'has', 'no', 'o', 't', 'how', 'needn', "isn't", "it's", 'themselves', 'during', 'which', 'and', 'an', 'won', 'shouldn', "she's", 'himself', 'any', 'being', 'whom', 'in', 'he', 'his', 'below', 'own', 'she', 'some', 'by', 'both', 'then', 'down', 'against', 'about', 'on', 'after', 'doesn', "aren't", "you're", 'wasn', 'ma', 'myself', 'am', 'yourselves', "couldn't", 'what', 'because', "don't", "shan't", "wasn't", 'very', "weren't", "doesn't", 'shan', 'their', 'nor', 'above', 'those', "mustn't", 'for', 'why', 'or', 'few', "shouldn't", 'that', 're', 'had', 'than', 'couldn', "hasn't", 'where', 'up', 'before', 'they', 'him', 'did', 'other', 'do', 'when', "haven't", 'herself', 'been', 'if', 'just', "won't", "mightn't", 'should', 'most', 'yours', 'of', 'through', 'under', "wouldn't", 'here', 'me', 'but', 'itself', "that'll", 'as', 'off', 'my', 'we', "you'll", 'to', 'from', 'd', 'o

In [3]:
print(sent_pre_1("people like this car.", True)) 

{'people': 1, 'like': 1, 'car': 1}


In [4]:
##       
print (WMD("people like this car.", "those guys enjoy driving that.", True))
print (WMD("implementation is quite time consuming", "those guys enjoy driving that", False))
print (WMD("people", "those guys enjoy driving that", True))
print (WMD("people", "those guys", True))
print (WMD("president talk chicago", "president speech illinois", True))

     pcost       dcost       gap    pres   dres   k/t
 0:  5.2346e+00  5.2346e+00  6e+01  4e+00  4e-01  1e+00
 1:  4.0716e+00  4.1724e+00  3e+00  3e-01  3e-02  2e-01
 2:  4.7335e+00  4.7543e+00  3e-01  4e-02  5e-03  3e-02
 3:  4.7456e+00  4.7460e+00  8e-03  1e-03  1e-04  6e-04
 4:  4.7468e+00  4.7468e+00  8e-05  1e-05  1e-06  6e-06
 5:  4.7468e+00  4.7468e+00  8e-07  1e-07  1e-08  6e-08
 6:  4.7468e+00  4.7468e+00  8e-09  1e-09  1e-10  6e-10
Optimal solution found.
4.7468361312141
     pcost       dcost       gap    pres   dres   k/t
 0:  5.4718e+00  5.4718e+00  1e+02  7e+00  5e-01  1e+00
 1:  3.9341e+00  4.1112e+00  6e+00  5e-01  4e-02  2e-01
 2:  5.1091e+00  5.1651e+00  8e-01  1e-01  7e-03  7e-02
 3:  5.1667e+00  5.1787e+00  2e-01  3e-02  2e-03  2e-02
 4:  5.1805e+00  5.1807e+00  5e-03  7e-04  5e-05  3e-04
 5:  5.1812e+00  5.1812e+00  5e-05  7e-06  5e-07  3e-06
 6:  5.1812e+00  5.1812e+00  5e-07  7e-08  5e-09  3e-08
Optimal solution found.
5.181171843856026
     pcost       dcost    